In [1]:
import pandas as pd

In [2]:
# CREATE WHOLE DF

In [3]:
data = pd.read_csv('weekdays/monday.csv', delimiter = ';', parse_dates=True)
data.head()

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices


In [4]:
data.shape

(4884, 3)

In [5]:
data.dtypes

timestamp      object
customer_no     int64
location       object
dtype: object

In [6]:
# Encode the customer nr and day into one variable
...

In [7]:
# Extract the hour from the timestamp
data["timestamp"] = pd.to_datetime(data["timestamp"])
data["hour"] = data["timestamp"].dt.hour
data["time"] = data["timestamp"].dt.time

In [8]:
# DIFFERENCE BETWEEN CHECKOUT-NO & CUST-NO:
check_c = set(data[data["location"]=="checkout"]["customer_no"].unique()) # number of checked out customers
all_c = set(data["customer_no"].unique()) # number of all customers
diff = all_c.difference(check_c) # difference between all & checked out

In [9]:
diff

{1430, 1433, 1437, 1439, 1440, 1441, 1443, 1445, 1446, 1447}

In [10]:
#FILL IN 'CHECKOUTS'
for cust in diff:
    data = data.append({"timestamp":"2019-09-02 22:00:00","customer_no":cust,
    "location":"checkout"}, ignore_index=True)

#check again for dtype of 'timestamp'
data["timestamp"] = pd.to_datetime(data["timestamp"])

In [11]:
#NEW COL FOR TIME_SPEND IN LOC
data['time_spend'] = data.sort_values(['customer_no','timestamp']).groupby('customer_no')['timestamp'].diff().shift(-1)

In [12]:
data.head(10)

,timestamp,customer_no,location,hour,time,time_spend
0,2019-09-02 07:03:00,1,dairy,7.0,07:03:00,0 days 00:02:00
1,2019-09-02 07:03:00,2,dairy,7.0,07:03:00,0 days 00:03:00
2,2019-09-02 07:04:00,3,dairy,7.0,07:04:00,0 days 00:02:00
3,2019-09-02 07:04:00,4,dairy,7.0,07:04:00,0 days 00:04:00
4,2019-09-02 07:04:00,5,spices,7.0,07:04:00,0 days 00:01:00
5,2019-09-02 07:04:00,6,spices,7.0,07:04:00,0 days 00:01:00
6,2019-09-02 07:04:00,7,spices,7.0,07:04:00,0 days 00:01:00
7,2019-09-02 07:04:00,8,fruit,7.0,07:04:00,0 days 00:03:00
8,2019-09-02 07:05:00,1,checkout,7.0,07:05:00,NaT
9,2019-09-02 07:05:00,5,checkout,7.0,07:05:00,NaT


In [13]:
data = data.set_index('timestamp').groupby('customer_no').resample('1min').fillna('ffill').drop(columns='customer_no')

In [14]:
data.head(10)

location  hour      time      time_spend
customer_no timestamp                                                    
1           2019-09-02 07:03:00     dairy   7.0  07:03:00 0 days 00:02:00
            2019-09-02 07:04:00     dairy   7.0  07:03:00 0 days 00:02:00
            2019-09-02 07:05:00  checkout   7.0  07:05:00             NaT
2           2019-09-02 07:03:00     dairy   7.0  07:03:00 0 days 00:03:00
            2019-09-02 07:04:00     dairy   7.0  07:03:00 0 days 00:03:00
            2019-09-02 07:05:00     dairy   7.0  07:03:00 0 days 00:03:00
            2019-09-02 07:06:00  checkout   7.0  07:06:00             NaT
3           2019-09-02 07:04:00     dairy   7.0  07:04:00 0 days 00:02:00
            2019-09-02 07:05:00     dairy   7.0  07:04:00 0 days 00:02:00
            2019-09-02 07:06:00  checkout   7.0  07:06:00             NaT

In [15]:
# NEW COL FOR NEXT LOC
data['next_loc'] = data['location'].shift(-1)
data.head(10)

location  hour      time      time_spend  \
customer_no timestamp                                                       
1           2019-09-02 07:03:00     dairy   7.0  07:03:00 0 days 00:02:00   
            2019-09-02 07:04:00     dairy   7.0  07:03:00 0 days 00:02:00   
            2019-09-02 07:05:00  checkout   7.0  07:05:00             NaT   
2           2019-09-02 07:03:00     dairy   7.0  07:03:00 0 days 00:03:00   
            2019-09-02 07:04:00     dairy   7.0  07:03:00 0 days 00:03:00   
            2019-09-02 07:05:00     dairy   7.0  07:03:00 0 days 00:03:00   
            2019-09-02 07:06:00  checkout   7.0  07:06:00             NaT   
3           2019-09-02 07:04:00     dairy   7.0  07:04:00 0 days 00:02:00   
            2019-09-02 07:05:00     dairy   7.0  07:04:00 0 days 00:02:00   
            2019-09-02 07:06:00  checkout   7.0  07:06:00             NaT   

                                 next_loc  
customer_no timestamp                      
1           2019-09-02 07:03:00     dairy  
            2019-09-02 07:04:00  checkout  
            2019-09-02 07:05:00     dairy  
2           2019-09-02 07:03:00     dairy  
            2019-09-02 07:04:00     dairy  
            2019-09-02 07:05:00  checkout  
            2019-09-02 07:06:00     dairy  
3           2019-09-02 07:04:00     dairy  
            2019-09-02 07:05:00  checkout  
            2019-09-02 07:06:00     dairy

In [16]:
#next_loc for 'checkout'
data.loc[(data.location == 'checkout'), 'next_loc'] = 'checkout'

In [17]:
# calculate transition matrix
transition_matrix = pd.crosstab(data['location'], #or 'before'
                                data['next_loc'],
                                normalize=0)

In [18]:
transition_matrix

next_loc,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.088776,0.746961,0.061634,0.050891,0.051739
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.203077,0.086538,0.050000,0.613462,0.046923
spices,0.143429,0.188301,0.173878,0.095353,0.399038


In [19]:
# check if the matrix is correct
transition_matrix.sum(axis=1)

location
checkout    1.0
dairy       1.0
drinks      1.0
fruit       1.0
spices      1.0
dtype: float64

In [20]:
data.head(20)

location  hour      time      time_spend  \
customer_no timestamp                                                       
1           2019-09-02 07:03:00     dairy   7.0  07:03:00 0 days 00:02:00   
            2019-09-02 07:04:00     dairy   7.0  07:03:00 0 days 00:02:00   
            2019-09-02 07:05:00  checkout   7.0  07:05:00             NaT   
2           2019-09-02 07:03:00     dairy   7.0  07:03:00 0 days 00:03:00   
            2019-09-02 07:04:00     dairy   7.0  07:03:00 0 days 00:03:00   
            2019-09-02 07:05:00     dairy   7.0  07:03:00 0 days 00:03:00   
            2019-09-02 07:06:00  checkout   7.0  07:06:00             NaT   
3           2019-09-02 07:04:00     dairy   7.0  07:04:00 0 days 00:02:00   
            2019-09-02 07:05:00     dairy   7.0  07:04:00 0 days 00:02:00   
            2019-09-02 07:06:00  checkout   7.0  07:06:00             NaT   
4           2019-09-02 07:04:00     dairy   7.0  07:04:00 0 days 00:04:00   
            2019-09-02 07:05:00     dairy   7.0  07:04:00 0 days 00:04:00   
            2019-09-02 07:06:00     dairy   7.0  07:04:00 0 days 00:04:00   
            2019-09-02 07:07:00     dairy   7.0  07:04:00 0 days 00:04:00   
            2019-09-02 07:08:00  checkout   7.0  07:08:00             NaT   
5           2019-09-02 07:04:00    spices   7.0  07:04:00 0 days 00:01:00   
            2019-09-02 07:05:00  checkout   7.0  07:05:00             NaT   
6           2019-09-02 07:04:00    spices   7.0  07:04:00 0 days 00:01:00   
            2019-09-02 07:05:00     dairy   7.0  07:05:00 0 days 00:12:00   
            2019-09-02 07:06:00     dairy   7.0  07:05:00 0 days 00:12:00   

                                 next_loc  
customer_no timestamp                      
1           2019-09-02 07:03:00     dairy  
            2019-09-02 07:04:00  checkout  
            2019-09-02 07:05:00  checkout  
2           2019-09-02 07:03:00     dairy  
            2019-09-02 07:04:00     dairy  
            2019-09-02 07:05:00  checkout  
            2019-09-02 07:06:00  checkout  
3           2019-09-02 07:04:00     dairy  
            2019-09-02 07:05:00  checkout  
            2019-09-02 07:06:00  checkout  
4           2019-09-02 07:04:00     dairy  
            2019-09-02 07:05:00     dairy  
            2019-09-02 07:06:00     dairy  
            2019-09-02 07:07:00  checkout  
            2019-09-02 07:08:00  checkout  
5           2019-09-02 07:04:00  checkout  
            2019-09-02 07:05:00  checkout  
6           2019-09-02 07:04:00     dairy  
            2019-09-02 07:05:00     dairy  
            2019-09-02 07:06:00     dairy